In [1]:
import os
import gc
import numpy as np
import pandas as pd

In [2]:
# change working directory
os.chdir(os.getcwd())

In [3]:
def date_adjustment(df):
    
    df['Date'] = pd.to_datetime(df['Date'])
    cols = ['Year', 'Month', 'Day']
    cols.extend(df.columns.tolist())
    df['Day'] = df['Date'].dt.day
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df.sort_values(by=['Date'], inplace=True)
    df = df.reindex(columns=cols)
    df.drop(['Date'], axis=1, inplace=True)
    
    return df

def new_df(df, instrument_name='SPY'):
    
    df_adj = pd.DataFrame()
    df_adj['Date'] = pd.to_datetime(df['Date'].values)
    df_adj[instrument_name] = df['Adj Close'].values
    #df_adj['SMA30'] = df_adj[instrument_name].rolling(window=30).mean()
    #df_adj['SMA100'] = df_adj[instrument_name].rolling(window=100).mean()
    
    return df_adj

def buy_sell(signal):
    # time complexity: O(n)
    n = len(signal)
    if n <= 1:
        return

    profit, i = 0, 0
    data = []
    
    while i < (n-1):
        while i < (n-1) and signal.iloc[i+1,1] <= signal.iloc[i,1]:
            i += 1
        if i == (n-1):
            break
        buy = i
        i += 1
        while i < n and signal.iloc[i,1] >= signal.iloc[i-1,1]:
            i += 1
        sell = i-1
        data.append([signal.iloc[buy,0], signal.iloc[sell,0],
                     signal.iloc[buy,1], signal.iloc[sell,1],
                     signal.iloc[sell,1] - signal.iloc[buy,1]])
    
    res = pd.DataFrame(data, columns=['Buy Date', 'Sell Date',
                                      'Buy Price', 'Sell Price',
                                      'Profit'])
    del data
    gc.collect()
    return res

def kadane_maxprofit(signal):
    # time complexity: O(n)
    # space complexity: O(1)
    profit = 0
    for i in range(1, len(signal)):
        if signal.iloc[i,1] > signal.iloc[i-1,1]:
            profit += signal.iloc[i,1] - signal.iloc[i-1,1]
    return profit

In [4]:
spy = pd.read_csv('./data/SPY (1).csv')
spy = new_df(spy)

In [5]:
spy

,Date,SPY
0,1993-03-01,25.637117
1,1993-03-02,26.017061
2,1993-03-03,26.125616
3,1993-03-04,25.980873
4,1993-03-05,25.908489
...,...,...
7385,2022-06-27,388.589996
7386,2022-06-28,380.649994
7387,2022-06-29,380.339996
7388,2022-06-30,377.250000


In [6]:
res = buy_sell(spy)
res

,Buy Date,Sell Date,Buy Price,Sell Price,Profit
0,1993-03-01,1993-03-03,25.637117,26.125616,0.488499
1,1993-03-05,1993-03-08,25.908489,26.487463,0.578974
2,1993-03-09,1993-03-10,26.397005,26.451286,0.054281
3,1993-03-12,1993-03-16,26.107527,26.234169,0.126642
4,1993-03-17,1993-03-18,26.071329,26.234169,0.162840
...,...,...,...,...,...
1865,2022-06-03,2022-06-07,408.774231,413.951843,5.177612
1866,2022-06-14,2022-06-15,372.261932,377.569031,5.307099
1867,2022-06-16,2022-06-21,365.072998,375.070007,9.997009
1868,2022-06-22,2022-06-24,374.390015,390.079987,15.689972


In [7]:
res['Profit'].sum()

3943.108112999999

In [8]:
maxg = kadane_maxprofit(spy)
maxg

3943.1081130000025